In [1]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
# 데이터 불러오고 합치기
#coin_dict = {}
#
#file_list = os.listdir("data")
#
#for f in file_list:
#    print(f)
#    df = pd.read_csv("Data/" + f, index_col=0,
#                     parse_dates=True)["close"]
#    coin_dict[str(f)[:-4]] = df
#
#coin_df = pd.DataFrame(coin_dict)

In [2]:
# 합친 데이터 불러오기
coin_df = pd.read_csv("all_coin_closeprice.csv", index_col=0,
                      parse_dates=True)

# 0은 NaN으로 변경
coin_df_with_nan = coin_df.replace(0,np.nan)

In [3]:
# 주간 리턴 계산
save_dict = {}
week_start  = pd.date_range("2010-01-01","2022-12-31", freq="W")

for i in range(len(week_start)-1):
    w_start = week_start[i]
    w_end = week_start[i+1] - pd.Timedelta(days=1)
    tmp_df = coin_df_with_nan.loc[w_start:w_end] # 일주일치 데이터 추출
    w_rtn = tmp_df.apply(lambda x: (x[-1] - x[0]) / x[0] + 1, axis=0) # 일주일간 리턴 계산
    save_dict[w_start] = w_rtn

weekly_return = pd.DataFrame(save_dict).T

In [10]:
# row별로 rank를 때려줌(수익률이 높을수록 0에 가까움) (NaN은 처리함)
weekly_return_rank = weekly_return.rank(axis=1, pct=True, 
                                        ascending=False, na_option="keep") 
weekly_return_rank

,0XBTC,1CR,1GOLD,1INCH,1SG,1ST,1WO,2015,2BACCO,2GIVE,...,ZSC,ZT,ZUC,ZUM,ZUR,ZUSD,ZXC,ZYN,ZYRO,ZYTARA
2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-20,NaN,0.491228,NaN,0.286875,0.634665,0.226771,0.578298,0.431287,0.783951,0.565140,...,0.931774,0.682261,0.783951,0.612248,0.783951,0.554906,NaN,0.051007,0.866472,NaN
2022-11-27,NaN,0.387914,NaN,0.937947,0.546946,0.125081,0.529565,0.552632,0.769493,0.479045,...,0.959389,0.663418,0.644574,0.316602,0.026641,0.418454,NaN,0.052632,0.080572,NaN
2022-12-04,NaN,0.339617,NaN,0.939227,0.437439,0.783555,0.346116,0.467013,0.568573,0.383653,...,0.021937,0.568573,0.568573,0.215632,0.568573,0.314917,NaN,0.970101,0.675008,NaN
2022-12-11,NaN,0.559740,NaN,0.842208,0.481331,0.728571,0.538636,0.557955,0.777273,0.415097,...,0.980519,0.163799,0.163799,0.513636,0.857792,0.432143,NaN,0.031169,0.662987,NaN


In [11]:
# rank를 기준으로 screening한 DataFrame을 만든다
high = weekly_return_rank.apply(lambda x: np.where(x < 1/3, 1, np.nan))
middle = weekly_return_rank.apply(lambda x: np.where(((x < 2/3) & (x >= 1/3)), 1, np.nan))
low = weekly_return_rank.apply(lambda x: np.where(x >= 2/3, 1, np.nan))

In [12]:
# 분위수별 주간 리턴 구하기(평균처줌)
high_rtn = (weekly_return * high).mean(skipna=True, axis=1)
middle_rtn = (weekly_return * middle).mean(skipna=True, axis=1)
low_rtn = (weekly_return * low).mean(skipna=True, axis=1)

In [46]:
# Sample기간 동안의 수익률 구하기 (기간을 길게 잡으니 inf가 뜸)
a = high_rtn.loc["2020":"2022"].cumprod()
b = middle_rtn.loc["2020":"2022"].cumprod()
c = low_rtn.loc["2020":"2022"].cumprod()

In [47]:
def do_regression(df, t):
    '''t만큼 시차를 가진 regression'''
    tmp_df = df.copy()
    y = tmp_df.shift(-t)
    concat = pd.concat([tmp_df, y], keys=["return","return_"+str(t)], axis=1) \
               .dropna()
    model = sm.OLS(concat["return_"+str(t)], concat["return"])
    result = model.fit(cov_type='HAC',cov_kwds={'maxlags':t}) #t-1인가?
    return result

In [48]:
low_1 = do_regression(a, 1)
low_2 = do_regression(a, 2)
low_4 = do_regression(a, 4)

middle_1 = do_regression(b, 1)
middle_2 = do_regression(b, 2)
middle_4 = do_regression(b, 4)

high_1 = do_regression(c, 1)
high_2 = do_regression(c, 2)
high_4 = do_regression(c, 4)
#rt1_1.params, rt1_1.pvalues

c:\Users\user\anaconda3\lib\site-packages\statsmodels\stats\sandwich_covariance.py:242: RuntimeWarning: overflow encountered in multiply
  xu = results.model.wexog * results.wresid[:, None]


In [51]:
pd.DataFrame({"R_t_t+1": [low_1.params[0], middle_1.params[0], high_1.params[0]],
              "R_t_t+2": [low_2.params[0], middle_2.params[0], high_2.params[0]],
              "R_t_t+4": [low_4.params[0], middle_4.params[0], high_4.params[0]]},
              index=["Low","Middle","Hogh"])

,R_t_t+1,R_t_t+2,R_t_t+4
Low,1.624224,2.380094,22.462557
Middle,0.995692,0.989949,0.981036
Hogh,0.882271,0.769769,0.565935
